### Test methods that work with accounts

In [1]:
from dexter.DB import DB, Account, Entry, Transaction, Document, Tag, Category, Column

import re

In [2]:
DB.init()

In [3]:
DB.open('dev')

### Match Account Names

In [4]:
Account.objects(name__contains='checking')

[<Account: <Ac assets:bank:checking assets checking>>]

In [5]:
Account.objects(name__contains='food')

[<Account: <Ac expenses:food expenses >>, <Account: <Ac expenses:food:groceries expenses groceries>>, <Account: <Ac expenses:food:restaurant expenses restaurant>>]

In [6]:
for a in DB.find_account('food'):
    print(a)

<Ac expenses:food expenses >
<Ac expenses:food:groceries expenses groceries>
<Ac expenses:food:restaurant expenses restaurant>


In [7]:
DB.account_name_parts()

{'assets',
 'bank',
 'car',
 'chase',
 'checking',
 'entertainment',
 'equity',
 'expenses',
 'food',
 'fuel',
 'groceries',
 'home',
 'household',
 'income',
 'interest',
 'liabilities',
 'mortgage',
 'payment',
 'restaurant',
 'savings',
 'utility',
 'visa',
 'yard',
 'yoyodyne'}

In [8]:
DB.account_name_parts('expenses')

set()

In [9]:
dct = DB.account_names()

In [10]:
dct

{'equity': {'equity'},
 'assets:bank:checking': {'assets:bank:checking'},
 'checking': {'assets:bank:checking'},
 'assets': {'assets:bank:checking', 'assets:bank:savings'},
 'bank': {'assets:bank:checking', 'assets:bank:savings'},
 'assets:bank:savings': {'assets:bank:savings'},
 'savings': {'assets:bank:savings'},
 'expenses:car': {'expenses:car'},
 'expenses': {'expenses:car',
  'expenses:car:fuel',
  'expenses:car:payment',
  'expenses:entertainment',
  'expenses:food',
  'expenses:food:groceries',
  'expenses:food:restaurant',
  'expenses:home',
  'expenses:home:household',
  'expenses:home:mortgage',
  'expenses:home:utility',
  'expenses:home:yard'},
 'car': {'expenses:car', 'expenses:car:fuel', 'expenses:car:payment'},
 'expenses:car:payment': {'expenses:car:payment'},
 'payment': {'expenses:car:payment'},
 'expenses:car:fuel': {'expenses:car:fuel'},
 'fuel': {'expenses:car:fuel'},
 'expenses:entertainment': {'expenses:entertainment'},
 'entertainment': {'expenses:entertainment'

In [11]:
dct['expenses']

{'expenses:car',
 'expenses:car:fuel',
 'expenses:car:payment',
 'expenses:entertainment',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'expenses:home:household',
 'expenses:home:mortgage',
 'expenses:home:utility',
 'expenses:home:yard'}

In [12]:
len(dct['expenses'])

12

In [13]:
dct = DB.account_names('expenses')

In [14]:
dct

{}

In [15]:
len(dct)

0

In [16]:
# DB.account_groups()

In [17]:
# DB.account_groups(['expenses'])

In [18]:
# DB.account_groups(['expenses:1'])

In [19]:
# DB.account_groups(['expenses:2'])

In [20]:
# DB.account_groups(['expenses:3'])

In [21]:
# DB.account_groups(['expenses:food'])

In [22]:
# DB.account_groups(['expenses:food:2'])

In [23]:
# DB.account_groups(['expenses:food:3'])

In [24]:
# DB.account_groups(['expenses','income'])

In [25]:
# DB.account_groups(['expenses:1','income:1'])

In [26]:
# DB.account_groups(['expenses:2','income:2'])

In [27]:
Account.objects(name__exact='expenses:food:groceries')

[<Account: <Ac expenses:food:groceries expenses groceries>>]

In [28]:
Account.objects(abbrev__exact='groceries')

[<Account: <Ac expenses:food:groceries expenses groceries>>]

In [29]:
s = 'expenses:food:groceries'

In [30]:
if acct := Account.objects(name__exact=s) or Account.objects(abbrev__exact=s):
    print(acct[0].name)

expenses:food:groceries


In [31]:
DB.fullname('expenses:food:groceries')

'expenses:food:groceries'

In [32]:
DB.fullname('groceries')

'expenses:food:groceries'

In [33]:
DB.fullname('groc')

In [34]:
Account.objects.get(name='assets:bank:checking')

<Account: <Ac assets:bank:checking assets checking>>

In [35]:
Account.objects.get(abbrev='checking')

<Account: <Ac assets:bank:checking assets checking>>

Bummer -- would be nice if this returned None instead of raising an exception:

In [36]:
# Account.objects.get(abbrev='foo')

In [37]:
Account.DoesNotExist

dexter.DB.DoesNotExist

In [38]:
Account.objects.first()

<Account: <Ac equity equity None>>

In [39]:
def abbrev(a):
    try:
        acct = Account.objects.get(name=a)
        res = acct.abbrev
    except Account.DoesNotExist:
        res = a
    return res

In [40]:
abbrev('assets:bank:checking')

'checking'

In [41]:
abbrev('assets:bank:whatever')

'assets:bank:whatever'

In [42]:
abbrev('expenses:personal:bob')

'expenses:personal:bob'

### Balances

In [43]:
DB.select(Entry, account='expenses:food*')

[<Entry: <En 2024-04-26 expenses:food:groceries +$15.0 []>>, <Entry: <En 2024-04-24 expenses:food:groceries +$15.0 []>>, <Entry: <En 2024-04-22 expenses:food:restaurant +$50.0 []>>, <Entry: <En 2024-04-23 expenses:food:groceries +$15.0 []>>, <Entry: <En 2024-04-01 expenses:food:restaurant +$70.0 []>>, <Entry: <En 2024-04-03 expenses:food:restaurant -$35.0 []>>, <Entry: <En 2024-04-12 expenses:food:groceries +$65.0 []>>, <Entry: <En 2024-04-02 expenses:food -$600.0 [<Tag.B: '#budget'>]>>]

In [44]:
for e in DB.select(Entry, account='expenses:food*'):
    print(e)

<En 2024-04-26 expenses:food:groceries +$15.0 []>
<En 2024-04-24 expenses:food:groceries +$15.0 []>
<En 2024-04-22 expenses:food:restaurant +$50.0 []>
<En 2024-04-23 expenses:food:groceries +$15.0 []>
<En 2024-04-01 expenses:food:restaurant +$70.0 []>
<En 2024-04-03 expenses:food:restaurant -$35.0 []>
<En 2024-04-12 expenses:food:groceries +$65.0 []>
<En 2024-04-02 expenses:food -$600.0 [<Tag.B: '#budget'>]>


In [45]:
for t in DB.select(Transaction, tag='#budget'):
    print(t)

<Tr 2024-04-02 expenses:car/expenses:entertainment/expenses:food/expenses:home -> income:yoyodyne $3700.0 Fill envelopes ['#budget']>
<Tr 2024-04-30 expenses:entertainment -> income:interest $2.27 Allocate interest ['#budget']>


In [46]:
DB.column_sum('food',Column.dr)

230.0

In [47]:
DB.column_sum('food',Column.cr)

635.0

In [48]:
DB.column_sum('food',Column.dr,nobudget=True)

230.0

In [49]:
DB.column_sum('food',Column.cr,nobudget=True)

35.0

In [50]:
DB.balance('food')

230.0
635.0


-405.0

In [51]:
DB.balance('food', nobudget=True)

230.0
35.0


195.0

In [52]:
DB.balance('food', ending='2024-04-10')

70.0
635.0


-565.0

In [53]:
DB.balance('food', ending='2024-04-10', nobudget=True)

70.0
35.0


35.0

In [54]:
DB.column_sum('food', Column.dr, starting='2024-04-01', ending='2024-04-12')

135.0

In [55]:
for e in Entry.objects(account__iregex='food', date__gte='2024-04-01', date__lte='2024-04-12'):
    print(e)

<En 2024-04-01 expenses:food:restaurant +$70.0 []>
<En 2024-04-03 expenses:food:restaurant -$35.0 []>
<En 2024-04-12 expenses:food:groceries +$65.0 []>
<En 2024-04-02 expenses:food -$600.0 [<Tag.B: '#budget'>]>


In [56]:
for e in DB.select(Entry, account='expenses:car'):
    print(e)

<En 2024-04-05 expenses:car:payment +$400.0 []>
<En 2024-04-02 expenses:car +$5.0 []>
<En 2024-04-03 expenses:car:fuel +$50.0 []>
<En 2024-04-02 expenses:car -$700.0 [<Tag.B: '#budget'>]>


In [60]:
for e in DB.select(Entry, account='car$'):
    print(e)

<En 2024-04-02 expenses:car +$5.0 []>
<En 2024-04-02 expenses:car -$700.0 [<Tag.B: '#budget'>]>


In [57]:
DB.select(Entry, account='expenses:car').sum('amount')

1155.0

In [ ]:
# Account(name='expenses:car:fuel:gas', category=Category.E).save()

In [ ]:
# Account(name='expenses:car:fuel:electric', category=Category.E).save()

In [ ]:
for a in Account.objects(category = Category.E):
    print(a.name)

In [ ]:
    def expand_node(node, level):
        '''
        A command line argument had an account name and level.  Return the
        list of accounts below to the specified level.
        '''
        res = []
        for acct in Account.objects(name__startswith=node):
            name = acct.name
            tail = name[len(node):]
            print(name, tail, tail.count(':'))
            if tail.count(':') <= level:
                res.append(name)
        return res

In [ ]:
expand_node('expenses:car', 1)

In [ ]:
DB.expand_node('expenses:car:1')

In [ ]:
DB.expand_node('expenses:car:2')

In [ ]:
DB.expand_node('expenses:car:0')

In [ ]:
expand_node('assets',1)

In [ ]:
re.match(r'.*:\d+', 'expenses:car:1')

In [ ]:
re.match(r'(.*):(\d+)','expenses:car:10').groups()

In [ ]:
DB.account_glob('expenses:car')

In [ ]:
DB.account_glob('groceries')

In [ ]:
DB.account_glob('expenses:car:')

In [ ]:
DB.account_glob('expenses:cart')

In [ ]:
DB.account_glob('expenses:car:1')

In [ ]:
DB.account_glob('expenses:car:2')

In [ ]:
DB.account_glob('expenses:car:0')

In [ ]:
DB.account_glob('@car')

In [ ]:
DB.account_glob('expenses:car:')

In [ ]:
DB.account_glob('expenses:car:0')

In [ ]:
def account_args(s):
    if s.startswith('@'):
        res = { 'account__regex': f'\\b{s[1:]}\\b' }
    elif s.endswith(':'):
        res = { 'account__regex': f'^{s[:-1]}.*$' }
    else:
        res = { 'account__regex': f'^{s}$'}
    return res

In [ ]:
account_args('expenses:car')

In [ ]:
account_args('expenses:car:')

In [ ]:
account_args('@car')

In [ ]:
e = account_args('expenses:')['account__regex']

In [ ]:
for a in Account.objects:
    if re.match(e, a.name):
        print(a.name)

In [ ]:
lst = Transaction.objects(pamount__lte=50)

In [ ]:
lst

In [ ]:
len(lst)

In [ ]:
type(lst)

In [ ]:
lst(pdate__lte='2024-04-03')

In [ ]:
for t in lst(pdate__lte='2024-04-03'):
    print(t)

In [ ]:
dct = {'a': 0, 'b': 1}

In [ ]:
# del dct['a']     <- works if 'a' in dict, raises exception if not
# dct.pop('a)      <- same
dct.pop('a', None)    # what we want -- returns None if 'a' not in dict

In [ ]:
dct

In [ ]:
lst1 = Transaction.objects(pamount__lte=50, pdate__lte='2024-04-03')

In [ ]:
for t in lst1:
    print(t)

In [ ]:
lst2 = Transaction.objects(pamount__lte=50, pdate__gte='2024-04-25')

In [ ]:
for t in lst2:
    print(t)

In [ ]:
# for t in lst1+lst2:     <- QuerySet is not list-like :-(
#     print(t)

In [ ]:
for e in Entry.objects:
    if e.tref is None:
        print(e)

In [ ]:
DB.account_glob('expenses:1')

In [ ]:
DB.account_glob('assets:1')

In [ ]:
DB.account_glob('groceries')